In [1]:
# Libraries for R^2 visualization
from ipywidgets import interactive, IntSlider, FloatSlider
from math import floor, ceil
from sklearn.base import BaseEstimator, RegressorMixin

# Libraries for model building
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Library for working locally or Colab
import sys

In [4]:
# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/bloominstituteoftechnology/DS-Unit-2-Applied-Modeling/master/data/'

# If you're working locally:
else:
    DATA_PATH = '../data/'

# I. Wrangle Data

In [61]:
def wrangle(filepath):
  #The Initial Data Read In
  df = pd.read_csv(DATA_PATH + 'elections/bread_peace_voting.csv')

  #Subjective Wrangling Variables (to be edited)
  high_cardinality_percentage_threshold = 0.6 #Very Subjective, Should Shrink as Dataset Gets Larger
  #high_cardinality_count_threshold = 10

  #Procedural Wrangling Variables
  cols_to_drop = [] #More Efficient To Only Call Drop Function Once
  identifier_cols = []

  #Formatting Column Names
  df.columns = df.columns.str.upper().str.replace(' ','_')

  #Find Indentifier Variables
  for col in df:
    if df[col].nunique() == df.shape[0]:
      identifier_cols.append(col)
  df = df.set_index(identifier_cols[0])
  #DEFAULTS TO FIRST IDENTIFIER
  #OTHER IDENTIFIERS WILL BE DELETED

  #Drop Columns With Only One Value
  for col in df:
      if df[col].nunique() == 1:
        cols_to_drop.append(col)

  #Drop Columns With High Cardinality (Too Many Categorical Variable States)
  for col in df:
    if df[col].dtype == 'object': #Filter For Categorical Variables
      if df[col].nunique() / df.shape[0] > high_cardinality_percentage_threshold:
        cols_to_drop.append(col)

  print('DROPPED COLUMNS:', cols_to_drop)
  df = df.drop(cols_to_drop, axis=1)
  return df

df = wrangle(DATA_PATH + 'elections/bread_peace_voting.csv')

DROPPED COLUMNS: ['INCUMBENT_PARTY_CANDIDATE', 'OTHER_CANDIDATE']


In [64]:
print(list(df.columns)); print()

print(df.nunique()); print()

#print(df.head()); print()

#print(df.info()); print()

['AVERAGE_RECENT_GROWTH_IN_PERSONAL_INCOMES', 'US_MILITARY_FATALITIES_PER_MILLION', 'INCUMBENT_PARTY_VOTE_SHARE']

AVERAGE_RECENT_GROWTH_IN_PERSONAL_INCOMES    16
US_MILITARY_FATALITIES_PER_MILLION            8
INCUMBENT_PARTY_VOTE_SHARE                   17
dtype: int64



If a model performs well on unseen data (the testing set), it is said to be **generalized**

**Overfitting** is when a **complex model** is being perfectly matched to the data, giving **low bias** and **high variance**

**Underfitting** is when a **simple model** is being roughly approximated to the data, giving **high bias** but **low variance**

**BIAS** the degree of matching to the data set

**VARIANCE** in how much the model changes when trained on different subsets of your data set

# II. Split Data

**First** we need to split our **target vector** from our **feature matrix**.

In [57]:
#THIS SHORTCUT REQUIRES PROPER WRANGLING
target = 'INCUMBENT_PARTY_VOTE_SHARE'
y = df[target]
X = df.drop(columns=target)
#X = df.drop(target,axis=1) #Alternative Method

**Second** we need to split our dataset into **training** and **test** sets.

Two strategies:

- Random train-test split using [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Generally we use 80% of the data for training, and 20% of the data for testing.
- If you have **timeseries**, then you need to do a "cutoff" split.
  * We want our train/test splits to preserve temporal order so that the model can demonstrate future prediction

In [73]:
from sklearn.model_selection import train_test_split

percentage_to_save_for_testing = 0.2

#RANDOM SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y,
                test_size=percentage_to_save_for_testing,
                random_state=42) #This Will Fix The Rank Seed For The Split

#CUTOFF SPLIT (used for time series)

cutoff = 2004

#print(np.percentile(np.array(X.index), 0.2)) 

mask = X.index < cutoff

X_train = X.loc[mask]
y_train = y.loc[mask]

X_test = X.loc[~mask]
y_test = y.loc[~mask]

#np.bitwise_not(mask) #What does this do? I wasn't paying attention.

# III. Establish Baseline

In [77]:
y_pred_baseline = [y_train.mean()] * len(y_train)
baseline_mae = mean_absolute_error(y_train, y_pred_baseline)

print('Mean Vote Share:', y_train.mean())
print('Baseline MAE:', baseline_mae)

Mean Vote Share: 52.56307692307692
Baseline MAE: 5.132544378698224


# IV. Build Model

In [78]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

# V. Check Metrics

## Mean Absolute Error

The unit of measurement is the same as the unit of measurment for your target (in this case, vote share [%]).

In [79]:
print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('Test MAE:', mean_absolute_error(y_test, model.predict(X_test)))

Training MAE: 1.3737002516876717
Test MAE: 1.4133453705003944


Is it generalizing well? The difference is relatively small, so the model is **generalizing somehwat well**

Is it overfit or underfit? **Slightly overfit** (works better on training set)

## Root Mean Squared Error

The unit of measurement is the same as the unit of measurment for your target (in this case, vote share [%]).

$ \sqrt{\frac{\sum{ (y-\hat{y})^2 } }{n}} $

In [83]:
print('Training RMSE:', mean_squared_error(y_train, model.predict(X_train), squared=False))
print('Test RMSE:', mean_squared_error(y_test, model.predict(X_test), squared=False))

Training RMSE: 2.0376005753896838
Test RMSE: 1.623765003537601


This shows that there are more outliers in the training set?

## $R^2$ Score

TL;DR: Usually ranges between 0 (bad) and 1 (good).

Equation for R2: 1 - (ModelSS)/(Baseline SS)

where SS = sum of squares

In [82]:
print('Training R2:', r2_score(y_train, model.predict(X_train)))
print('Test R2:', r2_score(y_test, model.predict(X_test)))

Training R2: 0.8759203630180873
Test R2: 0.4993519697110995


In [84]:
#BASED ON MODEL, .score WILL FIND THE APPROPRIATE SCORE ASSOCIATED WITH THAT MODEL
# For Linear Regressions, It Gives R2
print('Training R2:', model.score(X_train, y_train))
print('Test R2:', model.score(X_test, y_test))

Training R2: 0.8759203630180873
Test R2: 0.4993519697110995


**This looks bad, but we should always look at mean square errors in conjunction with R squared**

In [89]:
print(list(df.columns))

['AVERAGE_RECENT_GROWTH_IN_PERSONAL_INCOMES', 'US_MILITARY_FATALITIES_PER_MILLION', 'INCUMBENT_PARTY_VOTE_SHARE']


In [90]:
class BruteForceRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, m=0, b=0):
        self.m = m
        self.b = b
        self.mean = 0
        
    def fit(self, X, y):
        self.mean = np.mean(y)
        return self
    
    def predict(self, X, return_mean=True):
        if return_mean:
            return [self.mean] * len(X)
        else:
            return X * self.m + self.b

def plot(slope, intercept):
    # Assign data to variables
    x = df['AVERAGE_RECENT_GROWTH_IN_PERSONAL_INCOMES']
    y = df['INCUMBENT_PARTY_VOTE_SHARE']
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,6))
    
    # Set ax limits
    mar = 0.2
    x_lim = floor(x.min() - x.min()*mar), ceil(x.max() + x.min()*mar)
    y_lim = floor(y.min() - y.min()*mar), ceil(y.max() + y.min()*mar)
    
    # Instantiate and train model
    bfr = BruteForceRegressor(slope, intercept)
    bfr.fit(x, y)
    
    # ax1   
    ## Plot data
    ax1.set_xlim(x_lim)
    ax1.set_ylim(y_lim)
    ax1.scatter(x, y)
    
    ## Plot base model
    ax1.axhline(bfr.mean, color='orange', label='baseline model')
    
    ## Plot residual lines
    y_base_pred = bfr.predict(x)
    ss_base = mean_squared_error(y, y_base_pred) * len(y)
    for x_i, y_i, yp_i in zip(x, y, y_base_pred):
        ax1.plot([x_i, x_i], [y_i, yp_i], 
                 color='gray', linestyle='--', alpha=0.75)
    
    ## Formatting
    ax1.legend()
    ax1.set_title(f'Sum of Squares: {np.round(ss_base, 2)}')
    ax1.set_xlabel('Growth in Personal Incomes')
    ax1.set_ylabel('Incumbent Party Vote Share [%]')

    # ax2

    ax2.set_xlim(x_lim)
    ax2.set_ylim(y_lim)
    ## Plot data
    ax2.scatter(x, y)
    
    ## Plot model
    x_model = np.linspace(*ax2.get_xlim(), 10)
    y_model = bfr.predict(x_model, return_mean=False)
    ax2.plot(x_model, y_model, color='green', label='our model')
    for x_coord, y_coord in zip(x, y):
        ax2.plot([x_coord, x_coord], [y_coord, x_coord * slope + intercept], 
                 color='gray', linestyle='--', alpha=0.75)   
    
    ss_ours = mean_squared_error(y, bfr.predict(x, return_mean=False)) * len(y)
    
    ## Formatting
    ax2.legend()
    ax2.set_title(f'Sum of Squares: {np.round(ss_ours, 2)}')
    ax2.set_xlabel('Growth in Personal Incomes')
    ax2.set_ylabel('Incumbent Party Vote Share [%]')

y = df['INCUMBENT_PARTY_VOTE_SHARE']
slope_slider = FloatSlider(min=-5, max=5, step=0.5, value=0)
intercept_slider = FloatSlider(min=int(y.min()), max=y.max(), step=2, value=y.mean())
    
interactive(plot, slope=slope_slider, intercept=intercept_slider)

interactive(children=(FloatSlider(value=0.0, description='slope', max=5.0, min=-5.0, step=0.5), FloatSlider(va…

# VI. Communicate Results

**Challenge:** How can we find the coefficients and intercept for our `model`?

In [91]:
print("The Coefficient for Income is: ", model.coef_[0])
print("The Coefficient for Fatalities is: ", model.coef_[1])
print("The Model Intercept is: ", model.intercept_)

The Coefficient for Income is:  3.5769044691079976
The Coefficient for Fatalities is:  -0.05355141496518978
The Model Intercept is:  46.36824118234689


What is the most important feature?

Income because its coefficient has a greater absolute value

**Always use the coefficient magnitude to determine which features are most important**